In [ ]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scale = StandardScaler()
scale.fit(df_market_data)
df_market_data_scaled = scale.transform(df_market_data)

In [ ]:
# Create a DataFrame with the scaled data
df_scaled = pd.DataFrame(df_market_data_scaled)

# Copy the crypto names from the original data
df_scaled.columns = (df_market_data.columns.astype(str))

# Set the coinid column as index
df_scaled.index = df_market_data.index

# Display sample data
df_scaled.head(10)

---

### Find the Best Value for k Using the Original Data.

In [ ]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 11))


In [ ]:
# Create an empty list to store the inertia values
inertia_values = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=0, n_init='auto')
    kmeans.fit(df_market_data_scaled)
    inertia_values.append(kmeans.inertia_)


In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow = {"k": k_values, "inertia": inertia_values}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow)


In [35]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot = df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks = k
)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 4

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [ ]:
# Initialize the K-Means model using the best value for k
kmeans = KMeans(n_clusters=4, random_state=0, n_init='auto')


In [ ]:
# Fit the K-Means model using the scaled data
kmeans.fit(df_scaled)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
k_clusters = kmeans.predict(df_scaled)

# Print the resulting array of cluster values.
k_clusters

In [ ]:
# Create a copy of the DataFrame
df_clusters = df_scaled.copy()

In [ ]:
# Add a new column to the DataFrame with the predicted clusters
df_clusters['Cluster'] = k_clusters

# Display sample data
df_clusters.head(10)

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
scaled_plot = df_clusters.hvplot.scatter(
    x="price_change_percentage_24h", 
    y="price_change_percentage_7d", 
    by="Cluster", 
    hover_cols=["coin_id"],
    title="Cryptocurrency Clusters",
)

---

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [ ]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
pca_data = pca.fit_transform(df_scaled)

# View the first five rows of the DataFrame. 
pca_df_scaled = pd.DataFrame(data=pca_data, columns=['PC1', 'PC2', 'PC3'])
pca_df_scaled.index = df_scaled.index
pca_df_scaled.head()

In [ ]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
sum(pca.explained_variance_ratio_[:3])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 89.503%

In [ ]:
# Create a new DataFrame with the PCA data.
pca_df = pd.DataFrame(data=pca_data, columns=['PC1', 'PC2', 'PC3'])

# Set the coinid column as index
pca_df.index = df_market_data.index

# Display sample data
pca_df.head()

---

### Find the Best Value for k Using the PCA Data

In [ ]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 11))

In [ ]:
# Create an empty list to store the inertia values
inertia_values = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=0, n_init='auto')
    kmeans.fit(pca_df)
    inertia_values.append(kmeans.inertia_)


In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_pca = {"k": k_values, "inertia": inertia_values}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_pca = pd.DataFrame(elbow_pca)


In [37]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_pca_plot = df_elbow_pca.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k_values
)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No it does not
  

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [ ]:
# Initialize the K-Means model using the best value for k
kmeans = KMeans(n_clusters=4, random_state=0, n_init='auto')

In [ ]:
# Fit the K-Means model using the PCA data
kmeans.fit(pca_df)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the PCA data
pca_clusters = kmeans.predict(pca_df)
# Print the resulting array of cluster values.
pca_clusters

In [ ]:
# Create a copy of the DataFrame with the PCA data
pca_clusters_df = pca_df.copy()

# Add a new column to the DataFrame with the predicted clusters
pca_clusters_df['Cluster'] = pca_clusters

# Display sample data
pca_clusters_df.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
pca_plot = pca_clusters_df.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="Cluster",
    hover_cols=["coin_id"]
)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [38]:
# Composite plot to contrast the Elbow curves
elbow_plot + elbow_pca_plot


:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [ ]:
# Composite plot to contrast the clusters
# YOUR CODE HERE!
scaled_plot + pca_plot


#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Fewer features typically leads to more useful clustering. 